In [20]:
!pip install transformers
!pip install tensorflow-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [21]:
import tensorflow as tf
print(tf.test.gpu_device_name())

/device:GPU:0


In [22]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import TFRobertaForSequenceClassification, RobertaTokenizer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder

In [23]:
# Configure TensorFlow to use the GPU
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [24]:
# Load the concatenated dataset
df = pd.read_csv('normydata.csv')

In [25]:
# Split the data into training and validation sets
# train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['emotion'], random_state=42)
train_df = pd.read_csv('train.csv')
val_df = pd.read_csv('val.csv')

In [26]:
le = LabelEncoder()
train_df['emotion_encoded'] = le.fit_transform(train_df['emotion'])
val_df['emotion_encoded'] = le.transform(val_df['emotion'])

In [ ]:
train_df

In [27]:
# Get the class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_df['emotion_encoded']), y=train_df['emotion_encoded'])

# Convert the class weights to a dictionary
class_weights_dict = dict(enumerate(class_weights))


In [28]:
# Load the pre-trained RoBERTa tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-large')
model = TFRobertaForSequenceClassification.from_pretrained('roberta-large', num_labels=8)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
# Define the optimizer and loss function
optimizer = Adam(learning_rate=1e-5)
loss = SparseCategoricalCrossentropy(from_logits=True)

In [30]:
# Define the batch size
batch_size = 16

# Define the number of epochs
epochs = 10

# Define the learning rate schedule
def lr_schedule(epoch):
    if epoch < 3:
        return 1e-5
    else:
        return 1e-6

In [31]:
# Define the early stopping criteria
early_stopping = EarlyStopping(monitor='val_loss', patience=2)

In [32]:
# Tokenize the input data
train_encodings = tokenizer(train_df['text'].tolist(), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_df['text'].tolist(), truncation=True, padding=True, max_length=128)

In [33]:
# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_df['emotion_encoded'].tolist()
)).shuffle(len(train_df)).batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_df['emotion_encoded'].tolist()
)).batch(batch_size)

In [ ]:
# Train the model
with tf.device('/GPU:0'):
  model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
  sensei = model.fit(train_dataset, 
                      validation_data=val_dataset, 
                      class_weight=class_weights_dict,
                      epochs=epochs, 
                      callbacks=[early_stopping, LearningRateScheduler(lr_schedule)])

Epoch 1/10
 1318/10990 [==>...........................] - ETA: 3:23:08 - loss: 1.5952 - accuracy: 0.3926

CREATION OF THE TEST VAL AND TRAIN SETS
```
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the data
df = pd.read_csv('normydata.csv')

# Split the data into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Split the train data into train and validation sets
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

# Save the train, validation, and test sets to CSV files
train_df.to_csv('train.csv', index=False)
val_df.to_csv('val.csv', index=False)
test_df.to_csv('test.csv', index=False)

```